In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
np.set_printoptions(precision=17)

import importlib
import utils
importlib.reload(utils);

# Functions $f$, $g$, $h$, $h'$, $h_i$, and $h_i'$.  

In [36]:
def f(x):
    return x**2 - 4

def fp(x):
    return 2*x 

def g(x):
    return x-2

def gp(x):
    return 1

def sign(x,case_zero=0):
    if x > 0:
        return 1
    if x < 0:
        return -1
    if x == 0:
        return case_zero
    
def h(x,e,d,cz,f,g):
    return f(x) / g(x)

def h_under(gamma,e,d,cz,f,g,*args):
    num_underflow = f(2+gamma) == ((2+gamma)**2 - gamma**2)-4 and gamma != 0
    den_underflow = g(2+gamma) == 0 and gamma != 0
    return num_underflow, den_underflow

def hp_q(x,e,d,cz,f,g,fp,gp):
    return (fp(x)*g(x) - f(x)*gp(x))/(g(x)**2)

def hp_p(x,e,d,cz,f,g,fp,gp):
    return fp(x)/g(x) - f(x)*gp(x)/(g(x)**2) 

# 2x(x-2) - (x^2 - 4)
# 2(2 + gamma)(2 + gamma - 2) - ((2 + gamma)^2 - 4)
# x^2 - 4x + 4 = (x-2)^2 
# gamma^2

def hp_under(gamma,e,d,cz,f,g,fp,gp,*args):
    # TODO: Expand out numerator and figure out which term drops first.
    num_underflow = fp(2+gamma) != gamma**2 #and gamma != 0 
    den_underflow = g(2+gamma)**2 == 0 and gamma != 0
    print("{:3e} {:4} {:4} {:15e} {:15e} {:15e} ".format(gamma, num_underflow, den_underflow, fp(2+gamma)*g(2+gamma) - f(2+gamma)*gp(2+gamma), g(2+gamma)**2,gamma**2))
    return num_underflow, den_underflow

#######################
######### h1 ##########
#######################

def h1(x,epsilon,delta,case_zero,f,g):
    fx = f(x)
    gx = g(x)
    if 2-delta < x < 2 + delta:
        return fx/(gx + sign(gx,case_zero)*epsilon)
    return fx/gx

def h1p_q(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2-delta < x < 2+delta:
        denom = (gx + sign(gx,case_zero)*epsilon)
        return (fpx*denom - fx*gpx)/denom**2
    return (fpx*gx - fx*gpx)/(gx**2)

def h1p_p(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2-delta < x < 2+delta:
        return fpx/(gx + sign(gx,case_zero)*epsilon) - fx*gpx/((gx + sign(gx,case_zero)*epsilon)**2)
    return fpx/gx - fx*gpx/(gx**2) 

#######################
######### h2 ##########
#######################

def h2(x,epsilon,delta,case_zero,f,g):
    fx = f(x)
    gx = g(x)
    if 2 - delta < x < 2 + delta:
        return fx/(sign(gx,case_zero)*epsilon)
    return fx/gx

def h2p_q(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2 - delta < x < 2 + delta:
        return fpx/(sign(gx,case_zero)*epsilon) 
    return (fpx*gx - fx*gpx)/(gx**2)

def h2p_p(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2 - delta < x < 2 + delta:
        return fpx/(sign(gx,case_zero)*epsilon) 
    return fpx/gx - fx*gpx/(gx**2)

#######################
######### h3 ##########
#######################

def h3(x,epsilon,delta,case_zero,f,g):
    fx = f(x)
    gx = g(x)
    if - epsilon < gx < epsilon:
        return fx/(sign(gx,case_zero)*epsilon)
    return fx/gx

def h3p_q(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < epsilon:
        return fpx/(sign(gx,case_zero)*epsilon) 
    return (fpx*gx - fx*gpx)/(gx**2)

def h3p_p(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < epsilon:
        return fpx/(sign(gx,case_zero)*epsilon) 
    return fpx/gx - fx*gpx/(gx**2)

#######################
######### h4 ##########
#######################

def h4(x,epsilon,delta,case_zero,f,g):
    fx = f(x)
    gx = g(x)
    if - epsilon < gx < 0:
        return fx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fx/epsilon
    return fx/gx

def h4p_q(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < 0:
        return fpx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fpx/epsilon
    return (fpx*gx - fx*gpx)/(gx**2)

def h4p_p(x,epsilon,delta,case_zero,f,g,fp,gp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < 0:
        return fpx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fpx/epsilon
    return fpx/gx - fx*gpx/(gx**2)

# Universal Parameters

In [37]:
S = 1
EPSILON = [1e-4,1e-8,1e-12]
DELTA = [1e-4,1e-8,1e-12]
CASE_ZERO = [-1,1]

H = [h,h1,h2,h3,h4]
HPQ = [hp_q, h1p_q, h2p_q, h3p_q, h4p_q]
HPP = [hp_p, h1p_p, h2p_p, h3p_p, h4p_p]

HEPS = [[None],EPSILON,EPSILON,EPSILON,EPSILON]
HDEL = [[None],DELTA,DELTA,[None],[None]]
HCZ = [[None]] + [CASE_ZERO]*3 + [[None]]
HNAN = [(4,"text")] + [None]*4
HPNAN = [(1,"text")] + [None]*4

gamma = utils.get_x(0,20,0.25,include_zero=True) 

# Analytic $h_i'$

In [40]:
plot_hp = lambda i, porq, pq: utils.plot(gamma,porq[i],utils.call_h,h_under,(f,g,fp,gp),HEPS[i],
                                         HDEL[i],HCZ[i],s=S,shift=-1,deriv=True,hindex=i,
                                         yticks=utils.get_x(0,18,6),plot_nan=HPNAN[i],
                                         save=os.path.join(SAVE_LOC,"hp_removable"),file_start=pq)

for i in range(0):
    plot_hp(i,HPQ,"q")
    plot_hp(i,HPP,"p")



/Users/trevor/Documents/repos/autodiff_bad/code_trevor/utils.py:220: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  ax.scatter(x[loc],y[loc],c=col,s=s,label=l,**kwargs)


# Pytorch and Tensorflow $h'$

In [41]:
plot_hp_pkg = lambda i,ch,pkg: utils.plot(gamma,H[i],ch,h_under,(f,g),HEPS[i],HDEL[i],HCZ[i],s=S,
                                          shift=-1,deriv=True,hindex=i,yticks=utils.get_x(0,18,6),
                                          plot_nan=HPNAN[i],
                                          save=os.path.join(SAVE_LOC,f"hp_removable_{pkg}"))

for i in range(0):
    plot_hp_pkg(i,utils.call_h_torch,"torch")
    plot_hp_pkg(i,utils.call_h_tf,"tensorflow")


# Figuring out Jitter on $h'(x)$ Analytic and TF/Torch

In [42]:
ng,pg = utils.get_x(7,7.824,0.001,include_zero=False,mirror=False) 

header = ["$\\gamma$","$2+\\gamma$","$(2+\\gamma)^2$","$f(x)$","$f'(x)$","$g(x)$","$f'(x)/g(x)$","$g(x)^2$","$f(x)g'(x)/g(x)^2$","$h(x)$","$h'(x)-1$"]
spacing = "ccc|ccc|ccc|cc"

def hp_p_table(g):
    x = 2 + g
    x2 = np.power(x,2)
    fx = np.power(x,2)-4
    fpx = 2*x
    gx = x-2
    fpx_gx = fpx/gx
    gx2 = np.power(gx,2)
    fx_gx2 = fx/gx2
    hx = fx/gx
    hpx = fpx_gx - fx_gx2 - 1
    return np.stack((g,x,x2,fx,fpx,gx,fpx_gx,gx2,fx_gx2,hx,hpx),axis=1)




#print(array_to_tabular(hp_p_table(ng),header,spacing))

In [74]:
step = 0.0000000001
start = 7.000001
ng,pg = utils.get_x(start + step*839098,start + step*839098,step,include_zero=False,mirror=False) 
# TODO FINISH THIS
with open("test.csv",'w') as f:
    data = hp_p_table(ng)
    data = np.concatenate((data,[data[1,:]-data[0,:]]))
    f.write(array_to_csv(data,header))
with open("test.tex","w")as f:
    data = hp_p_table(ng)
    data = np.concatenate((data,[data[1,:]-data[0,:]]))
    f.write(array_to_tabular_T(data.T,header,"c|cc"))